<a href="https://colab.research.google.com/github/tamiresco/Spike-Timing/blob/master/Timing_Part_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Introdução

* Data Frame 1: Dados (quase) originais
* Data Frame 2: Dados organizados por distribuições de intervalos

Envirolment setting up

In [2]:
# basicos
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import stats

# google colab
from google.colab import files
from google.colab import drive
drive.mount('/content/drive/')

# codigo reyes
!git clone -l -s git://github.com/mbreyes/drrd.git clonedrepo
from clonedrepo.python import drrdTools

Mounted at /content/drive/
Cloning into 'clonedrepo'...
remote: Enumerating objects: 126, done.
remote: Total 126 (delta 0), reused 0 (delta 0), pack-reused 126
Receiving objects: 100% (126/126), 122.21 KiB | 15.28 MiB/s, done.
Resolving deltas: 100% (44/44), done.
     |████████████████████████████████| 61kB 1.9MB/s 


### Data Frame 1 - Funções

In [ ]:
def readFile (rats, sessions):

  # variaveis q armazenaram o nome dos ratos, os dados de cada rato e de qual trial sao esses dados
  ratsNumbers = []
  ratsData = []
  ratsTrials = []
  error = []

  # armazenando dados de cada rato
  for i in rats: 
    for j in sessions:

      try:
        df = pd.DataFrame(data= drrdTools.drrd(prefix='AB1',animalID=i,sessions=[j],plotFlag=True, dataPath='/content/drive/My Drive/ratinhos/'))
        ratsData.append(df)
        trial = 0

        # armazenando numero do rato para quando combinar os dados saber quais linhas sao de qual rato
        for row in df.iterrows():
          ratsNumbers.append(i)
          ratsTrials.append(trial)
          trial += 1

      except: 
        error.append((i,j))
        pass
  
  return ratsNumbers, ratsData, ratsTrials, error

In [ ]:
def createDataFrame1 (rats, sessions):

  ratsNumbers, ratsData, ratsTrials, error = readFile(rats, sessions)

  # juntando os dados de cada rato em um data frame só
  dataFrame1 = pd.concat(ratsData)

  # dando nome as colunas
  columns = ['press', 'notPress', 'success', 'valid', 'target', 'session'] 
  dataFrame1.columns = columns

  # adicionando a coluna do numero do rato e numero do trial
  dataFrame1['rat'] = ratsNumbers
  dataFrame1['trial'] = ratsTrials

  return dataFrame1, error

def createStructureDataFrame2 (dataFrame1):

  # para criar o segundo data frame:
  # selecionando os primeiros trials para depois retirarmos todas as linhas que não sao do primeiro trial e termos só a sessão
  indexFirstTrial = dataFrame1[ dataFrame1['trial'] != 0 ].index
  dataFrame2 = dataFrame1.drop(indexFirstTrial)

  # no data frame 2 retiramos as colunas 'press', 'notPress', 'sucess', 'valid' e tbm fazendo a transformação para indexadores
  dataFrame2.drop(['press', 'notPress', 'success', 'valid', 'trial','target' ], axis=1, inplace = True)
  dataFrame2.set_index(['rat', 'session'], inplace = True) 

  return dataFrame2

def cleanDataFrame1(dataFrame1): 

  # Agora podemos tirar no data frame 1 os invalidos e colocar os ratos, sessoes e trials como idexadores ao inves de colunas
  dataFrame1.drop(dataFrame1[dataFrame1.valid == 0].index, inplace=True)
  dataFrame1.set_index(['rat', 'session', 'trial'], inplace = True) 
  dataFrame1.drop(['valid', 'target'], axis=1, inplace = True)

  # retirando NaN
  # TODO rever a abrangencia do intervalo ao tirar uma linha que possivelmente ditaria o fim de um intervalo
  # Posso substituir o NaN pela media ao inves tbm
  dataFrame1.dropna(how='any', inplace=True)
  
  # TODO retirar outliers (tem dois jeitos: tradicional z-score ou sklearn.ensemble IsolationForest)





### Data Frame 2 - Funções


In [ ]:
def distribuitions(dataFrame1, dataFrame2):
  ''' As distribuições usuais sao: de duracao das respostas (T) e as do intervalos entre respostas (ITI). 
      Alem delas distribuicoes de outros intervalos, como a duração apenas das respostas certas. 
      Abaixo, nos comentarios tem as definicoes dos outros tipos de intervalos.
      Lembrar que o intervalo é do tipo [a;b[, i.e., o  primeiro elemento do intervalo é incluido e o ultimo nao. '''

  init = [[] for i in range(len(sessions) * len(rats))] #index de ratos * sessoes

  # Distribuições com numero de itens igual ao numero de trials
  T = [] # intervalos de duracao das respostas
  ITI = [] # intervalos entre respostas 

  # Distribuições com numero de itens igual ao numero de trials corretos
  A = [] # intervalos de duracao das respostas certas
  B = init # intervalos entre resposta certa e resposta certa 
  C = init # intervalos entre resposta certa e qualquer resposta = intervalos entre recompensa e qualquer resposta
  D = init # intervalos entre recompensa e recompensa = intervalos entre resposta certa e recompensa

  # Distribuições com numero de itens igual ao numero de trials incorretos
  E = [] # intervalos de duracao das respostas erradas
  F = init # intervalos entre resposta errada e resposta errada 
  G = init # intervalos entre resposta errada e qualquer resposta

  # Demais distribuições
  H = init # intervalos entre resposta certa e resposta errada
  I = init # intervalos entre resposta errada e resposta certa 
  J = init # intervalos entre resposta errada e recompensa

  # esse loop é com o dataFrame2, q so tem ratos e sessoes no index, pq queremos a distribuicao de cada sessao de cada rato
  for i, rows in dataFrame2.iterrows():

    # pega todos os trials dessa sessao e desse rato criando um vetor p eles q é adcionado nas variaveis T, ITI, A e E
    # so fazems esses 4 tipos de intervalo agora pq os outros demandam outro tipo de loop
    T.append(dataFrame1.at[(i[0],i[1]),'press']) 
    ITI.append(dataFrame1.at[(i[0],i[1]),'notPress']) 
    A.append(dataFrame1[dataFrame1['success'] == 1].at[(i[0],i[1]),'press'])
    E.append(dataFrame1[dataFrame1['success'] != 1].at[(i[0],i[1]),'press'])
    
  return T, ITI, A, B, C, D, E, F, G, H, I, J


In [ ]:
def addColumns(dataFrame1, dataFrame2):
  ''' Adiciona ao data frame 2 como colunas as distribuicoes de intervalos das sessoes'''

  T, ITI, A, B, C, D, E, F, G, H, I, J = distribuitions(dataFrame1, dataFrame2)

  dataFrame2['T'] = T
  dataFrame2['ITI'] = ITI
  dataFrame2['A'] = A
  dataFrame2['B'] = B
  dataFrame2['C'] = C
  dataFrame2['D'] = D
  dataFrame2['F'] = F
  dataFrame2['E'] = E
  dataFrame2['G'] = G
  dataFrame2['H'] = H
  dataFrame2['I'] = I
  dataFrame2['J'] = J

  init = [0 for i in range(len(sessions) * len(rats))]
  dataFrame2['success'] = init
  dataFrame2['fail'] = init
  dataFrame2['rate success'] = init

  return dataFrame2

In [ ]:
def createAndAddData (dataFrame1, dataFrame2):
  ''' Gera os tipos de Intervalos e os adciona em suas respectivas distribuicoes, q são as colunas do dataFrame2 '''

  # criando variaveis que gauardaram o tempo de uma seuquencia de sucesso ou fracasso
  successSequence = 0; failSequence = 0

  # ja nesse loop usamos dataFrame1 q tem trials no index, pq a variaveis variam de acordo com os trials
  for i, rows in dataFrame1.iterrows():
    
    # esse ele acertou
    if dataFrame1.at[(i[0],i[1],i[2]),'success'] == 1:
  
      dataFrame2.loc[(i[0],i[1]),'success'] = dataFrame2.loc[(i[0],i[1]),'success'] + 1 # numero de successo por sessoes

      # é o primeiro trial valido
      if i[2] == min(dataFrame1.loc[(i[0],i[1])].index):     
        successSequence = successSequence + dataFrame1.at[(i[0],i[1],i[2]),'press'] + dataFrame1.at[(i[0],i[1],i[2]),'notPress']

      # é um dos trials seguintes ao primeiro valido
      else:
        # e o anterior ele acertou tbm
        if dataFrame1.at[(i[0],i[1],(i[2]-1)),'success'] == 1:
          dataFrame2.loc[(i[0],i[1]),'C'].append(dataFrame1.at[(i[0],i[1],(i[2]-1)),'notPress'])     
          successSequence = successSequence + dataFrame1.at[(i[0],i[1],i[2]),'press'] + dataFrame1.at[(i[0],i[1],i[2]),'notPress']

        # mas o anaterior ele errou
        else: 
          dataFrame2.loc[(i[0],i[1]),'B'].append(successSequence)
          dataFrame2.loc[(i[0],i[1]),'D'].append(successSequence +dataFrame1.at[(i[0],i[1],i[2]),'press'])
          dataFrame2.loc[(i[0],i[1]),'G'].append(dataFrame1.at[(i[0],i[1],(i[2]-1)),'notPress'])
          dataFrame2.loc[(i[0],i[1]),'I'].append(dataFrame1.at[(i[0],i[1],(i[2]-1)),'notPress'])
          successSequence = 0

    # esse ele errou
    else:

      dataFrame2.loc[(i[0],i[1]),'fail'] = dataFrame2.loc[(i[0],i[1]),'fail'] + 1 # numero de fracasso por sessoes

      # é o primeiro trial valido
      if i[2] == min(dataFrame1.loc[(i[0],i[1])].index):     
        failSequence = failSequence + dataFrame1.at[(i[0],i[1],i[2]),'press'] + dataFrame1.at[(i[0],i[1],i[2]),'notPress'] 

      # é um dos trials seguintes ao primeiro valido
      else:       
        # e o anterior ele errou tbm    
        if dataFrame1.at[(i[0],i[1],(i[2]-1)),'success'] != 1:
          dataFrame2.loc[(i[0],i[1]),'G'].append(dataFrame1.at[(i[0],i[1],(i[2]-1)),'notPress'])      
          failSequence = failSequence + dataFrame1.at[(i[0],i[1],i[2]),'press'] + dataFrame1.at[(i[0],i[1],i[2]),'notPress'] 

        # mas o anterior ele acertou
        else: 
          dataFrame2.loc[(i[0],i[1]),'F'].append(failSequence)
          dataFrame2.loc[(i[0],i[1]),'J'].append(failSequence + dataFrame1.at[(i[0],i[1],i[2]),'press'])
          dataFrame2.loc[(i[0],i[1]),'C'].append(dataFrame1.at[(i[0],i[1],(i[2]-1)),'notPress'])     
          dataFrame2.loc[(i[0],i[1]),'H'].append(dataFrame1.at[(i[0],i[1],(i[2]-1)),'notPress'])
          failSequence = 0

    dataFrame2.loc[(i[0],i[1]),'rate success'] = dataFrame2.loc[(i[0],i[1]),'success'] / ( dataFrame2.loc[(i[0],i[1]),'fail'] + dataFrame2.loc[(i[0],i[1]),'success'] ) 

  return dataFrame2

In [ ]:
def alterIndex (df):
  '''Adiciona mais uma camada de index e de nomes de colunas. Isso sera necessarios para a função Plots'''

  # pegando so o trecho do dataframe q é desse rato
  # df = df.loc(axis=0)[[rat], :]

  # ascrecentando index de 1 a 30 
  df['n'] = range(0,len(df))
  df.set_index(['n'], append=True, inplace = True)

  # ascrecentando nome das colunas de '1' a '12' 
  x = [str(i) for i in np.array(range(len(df.columns)))] # criando o index das colunas novo
  array = [df.columns, x] # unindo o index das colunas novo ao antigo
  df.columns = pd.MultiIndex.from_arrays(array) # fazendo a duas camadas de index das colunas 

def plots(df, dt = 0.1, bw_method = 'scott'):

  alterIndex(df)

  # inicializando a estrutura dos subplots
  cols = len(df.columns)
  rows = len(df.index)
  fig = plt.figure(figsize=[3*cols, 5*rows] , constrained_layout=True, clear=True, dpi = 200)
  gs = fig.add_gridspec(cols,rows)

  for i in df.index:
     for j in df.columns: 

       grid, pdf = kernelDensityEstimation(df.loc[i,j], dt, bw_method)
       fig.add_subplot(gs[int(j[1]), i[2]]).plot(grid, pdf) #plotar o grafico, o grid definirá o eixo x e o pdf o eixo y
       fig.add_subplot(gs[int(j[1]), i[2]]).set_title('Rat: ' + str(i[0]) + ' - Session: '+ str(i[1]) + ' - Interval: ' + str(j[0]) + '\n') 
  
  plt.savefig('ratinhos.png')

### Criar arquivos para download

dataframe1

In [ ]:
# transformando em csv por partes e baixando-as
dataFrame1.to_csv('dataFrame1_50_75.csv')
files.download('dataFrame1_50_75.csv')

# unindo as csv em um data frame
frames = [pd.read_csv('dataFrame1_10_50.csv'), pd.read_csv('dataFrame1_50_75.csv')]
dataFrame1 = pd.concat(frames)

# transformando em csv de novo e baixando-o
dataFrame1.to_csv('dataFrame1.csv')
files.download('dataFrame1.csv')

dataframe2

In [ ]:
min_rats = 10
max_rats = 75
size_groups = 5

rats_groups = [range(i,i+size_groups) for i in range(min_rats, max_rats, size_groups)] 
sessions = range(1,7)

i=0

for rats in rats_groups:

  # construindo o dataframe1
  dataFrame1, error = createDataFrame1(rats, sessions)
  dataFrame2 = createStructureDataFrame2(dataFrame1)
  cleanDataFrame1(dataFrame1)
  error

  # construindo o dataframe2
  dataFrame2 = addColumns (dataFrame1, dataFrame2)
  dataFrame2 = createAndAddData (dataFrame1, dataFrame2)

  # transformando em csv por partes e baixando-as
  dataFrame2.to_csv('dataFrame2_'+ str(i) +'.csv')
  i = i+1